In [15]:
# ==============================
# Standard Library Imports
# ==============================
from enum import Enum
import random

# ==============================
# Third-party Library Imports
# ==============================
import matplotlib.pyplot as plt
from IPython.display import display, Latex
from matplotlib.ticker import MultipleLocator
import numpy as np  # Original numpy
import scipy as sp
import pennylane as qml

# Pennylane numpy
import pennylane.numpy as pnp

In [16]:
# ==============================
# Setup for Quantum Computations
# ==============================
# PennyLane settings
coeffs = [-0.5]
obs = [qml.PauliZ(0)]
hamiltonian = qml.Hamiltonian(coeffs, obs)
dev = qml.device('default.mixed', wires=1)

In [17]:
# Global Variables
Tau_global = 5e-2   # Dephase tau
Paras_global = pnp.zeros(2)
Phi_global = 0
Gamma_ps = 0

class DataIndex(Enum):
    BEFORE = 0
    PHI = 0
    CFI = 1
    PARAS = 2
    THETA_X = 2
    PHI_Z = 3

In [18]:
def Dephase_factor(tau):
    """
    Calculate the dephasing factor (gamma) based on the provided tau value.

    Args:
        tau (float): Dephasing time constant.

    Returns:
        float: Calculated dephasing factor (gamma).
    """
    return 1 - np.exp(-2 * tau)

$$
\rho_{After_Hadamard} = 

\frac{1}{2} 

\begin{bmatrix}

1 & e^{(i\phi - \tau)} \\
e^{(-i\phi - \tau)} & 1

\end{bmatrix}

=

\frac{1}{2} 

\begin{bmatrix}

1 & e^{i\phi} \sqrt{1 - \gamma} \\
e^{-i\phi} \sqrt{1 - \gamma} & 1

\end{bmatrix}$$

$$

where,
\quad \gamma = 
1 - e^{-2 \tau}
$$

$$ 
e^{-t} = \sqrt{1 - \gamma}
$$

In [19]:
@qml.qnode(dev)
def circuit(phi):
    global Tau_global, Paras_global
    
    # Set dephase factor 
    gamma_dephase_in = Dephase_factor(Tau_global)  

    theta_x, phi_z = Paras_global

    qml.RX(pnp.pi/2, wires = 0)
    # qml.Hadamard(wires=0)

    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    qml.PhaseDamping(gamma_dephase_in, wires = 0)

    qml.RZ(phi_z, wires = 0)  # phi_z
    
    qml.RX(theta_x, wires = 0)  # theta_x
    
    return qml.density_matrix(wires = 0)

In [20]:
def Cost_function(paras):
    """ Calculate Classical-Fisher-Information for qnode(=Post_selection_Dephase).
    
    Args:
        paras (Numpy array): [theta_init, tau_1, tau_2, tau_d1, tau_d2, tau_d3]

    Returns:
        _type_: CFI with minus(-) sign.
    """
    
    global Paras_global, Phi_global
    Paras_global = paras
    phi = pnp.array([Phi_global])    
    
    CFI = qml.qinfo.classical_fisher(circuit)(phi[0])
    return -CFI

In [ ]:
Sweep_data = np.array([0, np.pi*4, 1e-2])  #1e-2 ~ (4pi + 1e-2)
Sweep_data_arange = np.arange(Sweep_data[0], Sweep_data[1], Sweep_data[2])

In [21]:
init_params = pnp.array([np.pi/3, np.pi/2])

num_steps = 2000
optimizer = qml.GradientDescentOptimizer(stepsize=1e-2)

for idx_phi, phi in enumerate()

for i in range(num_steps):
    init_params = optimizer.step(Cost_function, init_params)
    
print("\nOptimized parameters:", init_params)
print("Optimized value:", Cost_function(init_params))


Optimized parameters: [1.57079633 1.57079633]
Optimized value: [[-0.90483742]]
